In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler

In [2]:
def init_weight(input_len, map_size):
    return np.random.rand(map_size[0], map_size[1], input_len)

def distance(point, weights):
    return np.linalg.norm(point - weights, axis=2)

def update_weight(weights, point, winner, learning_rate, sigma):
    distance = np.linalg.norm(np.array(np.indices(weights.shape[:2])).T - winner, axis=2)
    influence = np.exp(-distance / (2 * sigma**2))
    delta_weights = learning_rate * influence[:, :, np.newaxis] * (point - weights)
    weights += delta_weights

In [3]:
def train_kohonen(data, map_size, num_epochs, learning_rate_initial, sigma_initial):
    input_len = data.shape[1]
    weights = init_weight(input_len, map_size)

    for epoch in range(num_epochs):
        learning_rate = learning_rate_initial * np.exp(-epoch / num_epochs)
        sigma = sigma_initial * np.exp(-epoch / num_epochs)

        for point in data:
            distances = distance(point, weights)
            winner = np.unravel_index(np.argmin(distances), distances.shape)
            update_weight(weights, point, winner, learning_rate, sigma)

    return weights

def show_kohonen(weights, data, target, num_epochs):
    winner_coordinate = np.array([np.unravel_index(np.argmin(distance(point, weights)), weights.shape[:2])
                                   for point in data]).T

    marker = ['o', 's', 'D']
    colors = ['r', 'g', 'b']

    plt.figure(figsize=(10, 10))
    for i, (x, y) in enumerate(winner_coordinate.T):
        plt.plot(x + 0.5, y + 0.5, marker[target[i]], markerfacecolor='None',
                 markeredgecolor=colors[target[i]], markersize=10, markeredgewidth=2)

    plt.title(f'Mapa Kohonena dla zbioru Iris po {num_epochs} krokach')
    plt.show()

In [4]:
#ladowanie iris
iris = datasets.load_iris()
data = iris.data
target = iris.target

In [5]:
from sklearn.model_selection import train_test_split

#podział
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=77)

In [6]:
from sklearn.linear_model import LinearRegression

#utworzenie modelu liniowej regresji
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [7]:
from sklearn.metrics import mean_squared_error, r2_score

#uruchomienie
y_pred = model.predict(X_test)

#oceny
plt.plot(model.loss_curve_)
print("Model liniowej regresji wielowymiarowej")
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R^2:", r2_score(y_test, y_pred))

AttributeError: 'LinearRegression' object has no attribute 'loss_curve_'